In [2]:
!pip install autoawq

  Obtaining dependency information for autoawq from https://files.pythonhosted.org/packages/e0/45/4ec85c794a9436f14b8ed408511300622f6fc4ca8ab58ec7e34ddf5f2496/autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata
  Using cached autoawq-0.1.0-cp310-cp310-manylinux2014_x86_64.whl.metadata (12 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
  Using cached accelerate-0.23.0-py3-none-any.whl.metadata (18 kB)
  Using cached lm_eval-0.3.0-py3-none-any.whl (178 kB)
  Using cached texttable-1.6.7-py2.py3-none-any.whl (10 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached attributedict-0.3.0-py3-none-any.whl (14 kB)
  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/bb/c3/6a06208ecf0934ecaf509b51c52a6cf688586f54ae81ac65c56124571494/protobuf-4.24.3-cp37-abi3-manylinu

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = 'stockmark/stockmark-13b'
quant_path = 'stockmark-13b-awq'
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4 }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100% 40/40 [28:39<00:00, 42.98s/it]


('stockmark-13b-awq/tokenizer_config.json',
 'stockmark-13b-awq/special_tokens_map.json',
 'stockmark-13b-awq/tokenizer.json')

In [1]:
%%time
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer
import torch

quant_path = 'stockmark-13b-awq'
quant_file = "pytorch_model.bin"

# Load model
model = AutoAWQForCausalLM.from_quantized(quant_path, quant_file, fuse_layers=True)
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

def chat(text, max_new_tokens=512):
    tokens = tokenizer(
        text,
        return_tensors='pt'
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=max_new_tokens
    )

    return tokenizer.decode(generation_output[0])

Replacing layers...: 100% 40/40 [00:03<00:00, 10.78it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 10.5 s, sys: 6.94 s, total: 17.5 s
Wall time: 14.9 s


In [2]:
%%time
print(chat("クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
クマは、アザラシが浜辺に横たわっているのを発見しました。
クマは、アザラシが浜辺に横たわっているのを発見しました。 彼はアザラシを家に連れて帰ることに決めました。
クマがアザラシを家に連れて帰ることに決めました。 それから、クマは彼の友達と一緒に家に帰ることにしました。
クマは、アザラシと、アザラシが浜辺に横たわっているのを発見しました。 クマは、アザラシを家に連れて帰ることに決めました。
クマは、アザラシが浜辺に横たわっているのを発見しました。 それから、クマは彼の友達と一緒に家に帰ることにしました。</s>
CPU times: user 2.42 s, sys: 46.9 ms, total: 2.47 s
Wall time: 2.47 s


In [3]:
%%time
print(chat("小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。
【中学受験】小4の算数「四則混合計算」の無料学習プリント(問題集・ワークシートドリル)です。
四則混合計算は、中学入試ではほぼ出題される頻出問題です。
計算が複雑で、苦手な子も多い単元ですが、四則混合計算に慣れておくことはとても大切です。
四則混合計算は、小学4年生で学習する単元です。
四則混合計算は、中学入試ではほぼ出題される頻出問題です。計算が複雑で、苦手な子も多い単元ですが、四則混合計算に慣れておくことはとても大切です。
四則混合計算は、計算が複雑で、苦手な子も多い単元ですが、四則混合計算に慣れておくことはとても大切です。
四則混合計算は、中学入試ではほぼ出題される頻出問題です。 計算が複雑で、苦手な子も多い単元ですが、四則混合計算に慣れておくことはとても大切です。</s>
CPU times: user 3.09 s, sys: 3.15 ms, total: 3.09 s
Wall time: 3.09 s


In [4]:
%%time
print(chat("豊臣秀吉について知っていることを教えてください。"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>豊臣秀吉について知っていることを教えてください。
豊臣秀吉が行った主な政策を教えてください。
豊臣秀吉が行った主な政策を教えてください。
豊臣秀吉が行った主な政策を教えてください。 1 刀狩り 2 太閤検地 3 刀狩り
豊臣秀吉が行った主な政策を教えてください。 刀狩り 太閤検地
豊臣秀吉は、刀狩りや太閤検地以外にも、様々な政策を行っています。
豊臣秀吉は、刀狩りや太閤検地以外にも、様々な政策を行っています。 1 刀狩り 2 太閤検地 3 刀狩り 4 太閤検地 5 刀狩り 6 太閤検地 7 刀狩り 8 太閤検地 9 刀狩り 10 太閤検地
豊臣秀吉について、小学生に分かりやすい文章で教えてください。
豊臣秀吉について、小学生に分かりやすい文章で教えてください。 1 刀狩り 2 太閤検地 3 刀狩り 4 太閤検地 5 刀狩り 6 太閤検地 7 刀狩り 8 太閤検地 9 刀狩り 10 太閤検地 11 刀狩り 12 太閤検地 13 刀狩り 14 太閤検地 15 刀狩り 16 太閤検地 17 刀狩り 18 太閤検地 19 刀狩り 20 太閤検地
豊臣秀吉は、刀狩りや太閤検地以外にも、様々な政策を行っています。 1 刀狩り 2 太閤検地 3 刀狩り 4 太閤検地 5 刀狩り 6 太閤検地 7 刀狩り 8 太閤検地 9 刀狩り 10 太閤検地 11 刀狩り 12 太閤検地 13 刀狩り 14 太閤検地 15 刀狩り 16 太閤検地 17 刀狩り 18 太閤検地 19 刀狩り 20 太閤検地
豊臣秀吉が行った政策の中で、最も有名な政策は何ですか?
豊臣秀吉が行った政策の中で、最も有名な政策は何ですか? 
CPU times: user 8.23 s, sys: 7.08 ms, total: 8.24 s
Wall time: 8.24 s


In [5]:
%%time
print(chat("自然言語処理とは野球に例えると"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>自然言語処理とは野球に例えると、バッティングマシーンのようなもの。 自然言語処理は、機械にデータを与えて、そのデータに応じた答えを出力します。 野球に例えると、バッティングマシーンのようなもの。 バッティングマシーンに、データを与えると、そのデータに応じた答えを出力してくれます。
1995年生まれ。神奈川県出身。 16歳の頃から、独学でプログラミングを始める。 18歳でWebサービスの開発を始め、2018年、AI系ベンチャー企業に新卒入社。 入社後は、主に機械学習を用いたWebアプリケーションの開発を行う。 2020年に独立し、現在はフリーランスのエンジニア。 仕事では、機械学習を用いたWebサービスの開発を行っています。 趣味は、読書と野球観戦。 好きな野球チームは、横浜DeNAベイスターズ。
【中学数学】2つの円の交点を求める問題
【中学数学】連立方程式を解く</s>
CPU times: user 2.9 s, sys: 4.53 ms, total: 2.9 s
Wall time: 2.9 s


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("stockmark/stockmark-13b", device_map="auto", torch_dtype=torch.bfloat16)

# If you use a T4 or V100 GPU, please load a model in 8 bit with the below code.
# To do so, you need to install `bitsandbytes` via `pip install bitsandbytes`.
# model = AutoModelForCausalLM.from_pretrained("stockmark/stockmark-13b", device_map={"": 0}, load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("stockmark/stockmark-13b")

def chat2(text, max_new_tokens=512):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7
        )

    output = tokenizer.decode(tokens[0], skip_special_tokens=True)
    print(output)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
%%time
print(chat2("自然言語処理を野球に例えて小学生向けに説明すると"))

自然言語処理とは、人間が日常的に使っている言語をコンピュータで処理するための技術である。この技術には、主に以下3つの分野が存在する。
・自然言語の解析: 人間が使う言葉(自然言語)に対して意味や構造を解析する技術。構文解析や意味解析、文脈解析などが含まれる。
・自然言語の生成: 自然言語をコンピュータに理解させるだけでなく、コンピュータが自らの言葉を生成することも含まれる。
・自然言語のコミュニケーション: 自然言語を使ってコンピュータと人間がコミュニケーションを行う技術。質問応答や機械翻訳、言語モデリング、自然言語生成などが含まれる。
自然言語処理の目的は、人間とコンピュータが自然言語を使って対話
None
CPU times: user 3min 48s, sys: 379 ms, total: 3min 48s
Wall time: 3min 48s


In [ ]:
%%time
print(chat2("小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。"))

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer

model_name_or_path = "mmnga/stockmark-13b-AWQ-calib-ja"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, safetensors=True)

def chat3(text, max_new_tokens=512):
    tokens = tokenizer(
        text,
        return_tensors='pt'
    ).input_ids.cuda()

    # Generate output
    generation_output = model.generate(
        tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        max_new_tokens=max_new_tokens
    )

    print(tokenizer.decode(generation_output[0]))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Replacing layers...: 100% 40/40 [00:03<00:00, 10.80it/s]


In [2]:
%%time
print(chat3("小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。
小学生向けの四則演算の問題を5問出してください。最初は問題だけ、次に解答のみをください。 回答者は、算数が得意な小学生でも解けるレベルの解答をください。 よろしくお願いします。
小3算数 足し算と引き算の問題です。 問題 10+8=12 10-12=-2 12-1=11 11-2=9 9-4=5 12-3=9 9-7=2 13-5=8 14-4=10 10-9=1 15-3=12 16-2=14 17-5=12 18-7=11 19-1=18 20-4=16 21-5=16 22-8=14 23-1=22 24-6=18 25-4=19 26-8=18 27-1=26 28-7=18 29-9=19 30-5=25 31-1=30 32-8=24 33-3=28 34-2=32 35-6=29 36-4=32 37-1=36 38-7=31 39-1=38 40-8=32 41-5=36 42-2=38 43-3=41 44-6=40 45-4=41 46-8=38 47-1=46 48-7=41 49-2=47 50-4=46 51-5=46 52-3=49 53-1=52 54-6=50 55-4=50 56-8=48 57-7=54 58-2=56 59-4=58 60-5=55 61-1=60 62-7=
None
CPU times: user 8.32 s, sys: 409 ms, total: 8.73 s
Wall time: 8.73 s


In [8]:
print(chat3("""
次の文章を読んで末尾の問に答えよ。ただし、以下の文章中にある情報以外を捏造して答えてはならない。文章中からわからない情報については、「わからない」と解答せよ。

```
スティーブ・ジョブズは、アップルの共同創設者であり、パソコンやスマートフォンなどの革新的な製品を送り出したことで知られる企業家です。

1955年にアメリカで生まれ、若いころからエレクトロニクスに強い関心を持っていました。1970年代にスティーブ・ウォズニアックと出会い、ガレージでApple IやApple IIなどの初期のパソコンを開発しました。

その後もMacintoshの開発などを率い、使いやすさを追求した画期的な製品を次々と生み出しました。一時期アップルから離れていましたが、1990年代後半にCEOとしてアップルに復帰。その後iMacやiPod、iPhone、iPadなどのヒット商品を生み出し、アップルを世界的な企業に成長させました。

常にシンプルで洗練されたデザインを心がけ、「1,000のNOを言って、1つのYESを言う」という哲学で製品開発を行ったことでも知られています。 2011年に肝臓癌のため死去しましたが、その功績は極めて大きく、世界のテクノロジー業界に多大な影響を与えました。
```

問A ジョブスは何を開発しましたか？
問B ジョブスはアップル社のCTOでしたか？
問C ジョブスが癌となった時に利用した治療方法を答えよ
問D この文章中に登場する人物名をすべて答えよ
問E この文章中に登場する固有名詞をすべて答えよ
問F この文章の感情トーン分析をしてください
問G この文章は何段落ですか？
問H この文章を更にシンプルに要約してください
問I ジョブスが死去してから何年経ちましたか？

では以下がA~Iまでの解答になります。

答A
""", 1024))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>
次の文章を読んで末尾の問に答えよ。ただし、以下の文章中にある情報以外を捏造して答えてはならない。文章中からわからない情報については、「わからない」と解答せよ。

```
スティーブ・ジョブズは、アップルの共同創設者であり、パソコンやスマートフォンなどの革新的な製品を送り出したことで知られる企業家です。

1955年にアメリカで生まれ、若いころからエレクトロニクスに強い関心を持っていました。1970年代にスティーブ・ウォズニアックと出会い、ガレージでApple IやApple IIなどの初期のパソコンを開発しました。

その後もMacintoshの開発などを率い、使いやすさを追求した画期的な製品を次々と生み出しました。一時期アップルから離れていましたが、1990年代後半にCEOとしてアップルに復帰。その後iMacやiPod、iPhone、iPadなどのヒット商品を生み出し、アップルを世界的な企業に成長させました。

常にシンプルで洗練されたデザインを心がけ、「1,000のNOを言って、1つのYESを言う」という哲学で製品開発を行ったことでも知られています。 2011年に肝臓癌のため死去しましたが、その功績は極めて大きく、世界のテクノロジー業界に多大な影響を与えました。
```

問A ジョブスは何を開発しましたか?
問B ジョブスはアップル社のCTOでしたか?
問C ジョブスが癌となった時に利用した治療方法を答えよ
問D この文章中に登場する人物名をすべて答えよ
問E この文章中に登場する固有名詞をすべて答えよ
問F この文章の感情トーン分析をしてください
問G この文章は何段落ですか?
問H この文章を更にシンプルに要約してください
問I ジョブスが死去してから何年経ちましたか?

では以下がA~Iまでの解答になります。

答A
ジョブ


、
、?っ次文あるよんでをの末っ� っ�が、とのをにとの答を、答え、 っ �はば 、、、をの、
<s>ふ] フ.をでに関する(((とを、ーーーーーーーーでを�をねでにを狂をっつっにしくや楽は
は、、とのっ、からからっっじ月号までわからないがとの情報については、、 <s>、
1.
はとや」
とととと、
。<s>。。。




<s> 
っ�「,
`
<s>

<s>はっっ点章が、(2はびは々がっ(、<s>なた
っ�、のねは

In [10]:
print(chat3("""
スティーブ・ジョブズは、アップルの共同創設者であり、
""", 256))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<s>
スティーブ・ジョブズは、アップルの共同創設者であり、
iPhoneっっっっっ�っっっっっっ�っっっっっっっっっっっっっっっっっっっっっ�っ�っっっっっ�っ�っっっっっっ�っっっっ�っっっ�っ�っ�っっっっっっっぜいっげきっっっっっっっっっっっっっ�っっっっっっっっっっっっっ�っつかんっっっっっっっっっキャンペっっいついっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっ日程などっ�っっっわちっっっげきっ�っ�っ日程などっっっっげきっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっっくりっっっっっっっげきっっっっっ�っ�っっっっっ�っ�っ�っ�っ�っ�っっっっっっ�っ�っ�っぼっっっっっっ
None
